### Genetic Programming - Revised Model (Parameter Grid)
Revised implementation of GP using tree depth of 2 to 3 and basic operators. Strongly typed GP is used to ensure final tree node produces required signal string. Fitness function maximises risk-adjusted return and uses pandas apply to increase speed.. Uses robust scaled data compiled by industry with min. 80% complete data.


In [ ]:
import operator
import math
import random
import pandas as pd
import datetime as dt
import numpy as np
import pygraphviz as pgv
import os
import urllib
import json
import requests
from io import StringIO
import sys
from pathlib import Path
from eod import EodHistoricalData
from wand.image import Image as WImage

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

from sklearn.model_selection import ParameterGrid
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import plot_precision_recall_curve


pd.set_option('display.max_rows', None)
sys.set_int_max_str_digits(30000) 

In [5]:
#paths for data - set prefix to location of Data folder
path_prefix = r'D:'
path_att = r'\Data\fundamentals_by_attribute'
path_fun = r'\Data\fundamentals_by_ticker'
path_std = r'\Data\standardised_fundamentals'
path_eda = r'\Data\exploratory_data_analysis'
path_rob = r'\Data\robust_scaling'
path_ind = r'\Data\indicator_prices'
path_sec = r'\Data\rob_scaled_data_by_sector'
path_cor = r'\Data\correlation_matrices'
path_img = r'D:\GP_Images'

In [10]:
#read csv of % NaN values for each attribute and create list of those with more than given % for removal
mean_pct_nans = pd.read_csv(path_prefix + path_eda + "/df_agg.csv", index_col=0).loc['pct_nan']
mean_pct_nans.sort_values(inplace=True)
to_drop = mean_pct_nans[mean_pct_nans > 15].index.to_list()

In [11]:
#read example fundamental dataset for extracting column names
filepath = Path(path_prefix + path_fun + '\df_fun_TECK.csv')
df = pd.read_csv(filepath, index_col=0, parse_dates=True)
df.drop(['date', 'filing_date', 'currency_symbol'], inplace=True)
df.drop(to_drop, inplace=True)

In [12]:
#create list of types for when iniitialising Pset (float x num attributes)
num_atts = len(df)
types_list = []
for i in range(num_atts):
    types_list.append(float)

In [13]:
#initialise Pset to receive floats and output string (buy/sell/hold)
pset = gp.PrimitiveSetTyped("main", types_list, str)

In [14]:
#rename input arguments (Terminals) with fundamental attribute names
for i in range(num_atts): 
    ind_name =  df.index[i]
    if ind_name != 'date' and ind_name != 'currency_symbol' and ind_name != 'filing_date' :
        argstring = "ARG{}".format(str(i))
        pset.renameArguments(**{argstring:ind_name})

In [15]:
# Define new functions
def protectedDiv(left, right):
    try:
        return left / right
    except ZeroDivisionError:
        return left

def negate(left, right):
    try:
        return np.negative(left) * right
    except ZeroDivisionError:
        return left

def mult2(left, right):
    return left * 2 * right

def mult3(left, right):
    return left * 3 * right

def mult5(left, right):
    return left * 5 * right

def mult10(left, right):
    return left * 10 * right

def mult50(left, right):
    return left * 50 * right

def div2(left, right):
    return protectedDiv(left, 2) * right

def div3(left, right):
    return protectedDiv(left, 3) * right

def div5(left, right):
    return protectedDiv(left, 5) * right

def div10(left, right):
    return protectedDiv(left, 10) * right

def div50(left, right):
    return protectedDiv(left, 50) * right


In [16]:
#define function to provide signal
def buy_sell (A, B):
    if A > B:
        return True
    elif A < B:
        return False

In [17]:
#add functions to Primitive set
pset.addPrimitive(buy_sell, [float, float], str)
pset.addPrimitive(operator.add, [float, float], float)
pset.addPrimitive(operator.sub, [float, float], float)
pset.addPrimitive(operator.mul, [float, float], float)
pset.addPrimitive(protectedDiv, [float, float], float)

pset.addPrimitive(mult2, [float, float], float)
pset.addPrimitive(mult3, [float, float], float)
pset.addPrimitive(mult5, [float, float], float)
pset.addPrimitive(mult10, [float, float], float)
pset.addPrimitive(div2, [float, float], float)
pset.addPrimitive(div3, [float, float], float)
pset.addPrimitive(div5, [float, float], float)
pset.addPrimitive(div10, [float, float], float)
pset.addPrimitive(negate, [float, float], float)

In [18]:
#create invididual class to maximise fitness
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax, pset=pset)

In [19]:
#instantiate toolbox and register components for evolving trees
toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=2, max_=4)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

In [20]:
#define functions for performance 
def find_TP(actuals, signals):
   # counts the number of true positives (y = 1, y_hat = 1)
   return sum((actuals == True) & (signals == True))
def find_FN(actuals, signals):
   # counts the number of false negatives (y = 1, y_hat = 0) Type-II error
   return sum((actuals == True) & (signals == False))
def find_FP(actuals, signals):
   # counts the number of false positives (y = 0, y_hat = 1) Type-I error
   return sum((actuals == False) & (signals == True))
def find_TN(actuals, signals):
   # counts the number of true negatives (y = 0, y_hat = 0)
   return sum((actuals == False) & (signals == False))

In [21]:
def eval_trading_rule(individual):
    #transform the tree in to a callable function
    function = toolbox.compile(expr=individual)
    
    def gen_signal(row):
        vals = row.values.flatten().tolist()
        signal = function(*vals)
        return signal

    signals = X_train.apply(gen_signal, axis=1)
    actuals = y_train > 0.2
    
    accuracy = (actuals == signals).sum() / signals.shape[0]
    
    tree = gp.PrimitiveTree(individual)

    TP = find_TP(actuals, signals)
    FN = find_FN(actuals, signals)
    FP = find_FP(actuals, signals)
    TN = find_TN(actuals, signals)
    
    if TP > 0:
        precision = TP/(TP+FP)
        recall = TP/(TP+FN)
        f1_score = 2*((precision*recall)/(precision+recall))
        f0_5_score = ((1 + 0.5**2) * precision * recall) / (0.5**2 * precision + recall)
        
    else:
        precision = recall = f1_score = f0_5_score = 0
    
    return (f0_5_score, )

In [22]:
toolbox.register("evaluate", eval_trading_rule)
toolbox.register("select", tools.selDoubleTournament,  fitness_size=10, parsimony_size=1.6, fitness_first=True)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genHalfAndHalf, min_=2, max_=4)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

In [6]:
sectors = pd.read_csv(path_prefix + "/industries_by_ticker.csv", names=["Industry"])
sectors = sectors['Industry'].values
sectors = set(sectors[1:])

In [9]:
len(sectors)

162

In [24]:
param_grid = {'Crossover': [0.5, 0.9], 'Population': [100, 500], 'Mutation': [0.1, 0.5], 'Generations': [20, 50]}
list(ParameterGrid(param_grid))

[{'Crossover': 0.5, 'Generations': 20, 'Mutation': 0.1, 'Population': 100},
 {'Crossover': 0.5, 'Generations': 20, 'Mutation': 0.1, 'Population': 500},
 {'Crossover': 0.5, 'Generations': 20, 'Mutation': 0.5, 'Population': 100},
 {'Crossover': 0.5, 'Generations': 20, 'Mutation': 0.5, 'Population': 500},
 {'Crossover': 0.5, 'Generations': 50, 'Mutation': 0.1, 'Population': 100},
 {'Crossover': 0.5, 'Generations': 50, 'Mutation': 0.1, 'Population': 500},
 {'Crossover': 0.5, 'Generations': 50, 'Mutation': 0.5, 'Population': 100},
 {'Crossover': 0.5, 'Generations': 50, 'Mutation': 0.5, 'Population': 500},
 {'Crossover': 0.9, 'Generations': 20, 'Mutation': 0.1, 'Population': 100},
 {'Crossover': 0.9, 'Generations': 20, 'Mutation': 0.1, 'Population': 500},
 {'Crossover': 0.9, 'Generations': 20, 'Mutation': 0.5, 'Population': 100},
 {'Crossover': 0.9, 'Generations': 20, 'Mutation': 0.5, 'Population': 500},
 {'Crossover': 0.9, 'Generations': 50, 'Mutation': 0.1, 'Population': 100},
 {'Crossover

In [ ]:
hof_of_hof = []

for params in ParameterGrid(param_grid):

    #intialise GP function with parameter values from parameter grid
    def main():
        random.seed(41)
        pop = toolbox.population(n=params['Population'])
        hof = tools.HallOfFame(10)
        stats = tools.Statistics(lambda ind: ind.fitness.values)
        stats.register("avg", np.mean)
        stats.register("std", np.std)
        stats.register("min", np.min)
        stats.register("max", np.max)
        #stats.register("values", np.array)
    
        #algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 40, stats, halloffame=None)
        algorithms.eaSimple(pop, toolbox, params['Crossover'], params['Mutation'], params['Generations'], stats=stats, halloffame=hof)
        return pop, stats, hof


    for sector in sectors:
        if completed.count(sector) == 0:
            filepath = Path(path_prefix + path_sec + '\df_{}.csv'.format(sector))
    
            #load in fundamental sector data
            df_sec = pd.read_csv(filepath, index_col=0)
            df_sec.drop(to_drop, axis=1, inplace=True)
            df_sec.sort_index(inplace=True)
            
            
            #drop unecessary columns
            if 'adjusted_close' in df_sec.columns:
                df_sec.drop('adjusted_close', axis=1, inplace=True)
            
            if 'prev_close' in df_sec.columns:
                df_rob.drop('prev_close', axis=1, inplace=True)
            
            if 'signal' in df_sec.columns:
                df_sec.drop('signal', axis=1, inplace=True)
            
            #only apply to datasets > 10
            df_len = df_sec.shape[0]
            if df_len > 10:
                
                #create binary mask to split train and test sets 
                mask_train = 8
                mask_test = 2
                mask = mask_train * '1' + mask_test * '0'
                mask = mask * int((df_len - df_len % 10) / 10) + (df_len % 10) * '0'
                bin_mask = [int(d) for d in str(int(mask))]
                
                for i in range(len(bin_mask)):
                    if bin_mask[i] == 0:
                        bin_mask[i] = False
                 
                    if bin_mask[i] == 1:
                        bin_mask[i] = True
            
                X_train = df_sec.drop('10%DDret', axis=1)[bin_mask]
                X_test = df_sec.drop('10%DDret', axis=1)[np.invert(bin_mask)]
                y_train = df_sec['10%DDret'][bin_mask]
                y_test = df_sec['10%DDret'][np.invert(bin_mask)]
    
                #execute GP function
                print("*****EVOLUTION*****")
                if __name__ == "__main__":
                    pop, stats, hof = main()
    
                #apply best-performing function (hall-of-fame) to test set
                hof_func = gp.compile(hof[0], pset=pset)
            
                def gen_signal(row):
                    vals = row.values.flatten().tolist()
    
                    '''
                    for i in range(len(vals)):
                        if math.isnan(vals[i]) == True:
                            vals[i] = 0
                    '''
                    signal = hof_func(*vals)
                    return signal
            
                signals = X_test.apply(gen_signal, axis=1)
                actuals = y_test > 0.2
                tree = gp.PrimitiveTree(hof[0])
    
                #calculate performance metrics for test set
                TP = find_TP(actuals, signals)
                FN = find_FN(actuals, signals)
                FP = find_FP(actuals, signals)
                TN = find_TN(actuals, signals)
    
                #avoid div by 0 error 
                if TP > 0:
                    precision = TP/(TP+FP)
                    recall = TP/(TP+FN)
                    f1_score = 2*((precision*recall)/(precision+recall))
                    f0_5_score = ((1 + 0.5**2.0) * precision * recall) / (0.5**2.0 * precision + recall)
                    accuracy = (TP + TN) / (TP + FP + FN + TN)
                    
                else:
                    accuracy = precision = recall = f1_score = f0_5_score = 0
    
                if f0_5_score > 0.5:
                    hof_of_hof.append(hof)
                
                print("*****TEST_SET*****")
                print("Sector: = " + str(sector))
                print("accuracy = " + str(accuracy))
                print("precision = " + str(precision))
                print("recall = " + str(recall))
                print("f1_score = " + str(f1_score))
                print("f0.5_score = " + str(f0_5_score))
                print("Best performing rule = " + str(tree))
                print("Size training data: " + str(X_train.shape))
                print("Size test data: " + str(X_test.shape))
                
                record = stats.compile(pop)   
                
                with open(path_prefix + 'CV_Grid_Results_IV.txt', 'a') as f:
                    f.write('\n') 
                    f.write('\n')
                    f.write('\n')
                    f.write("**********")
                    f.write('\n')
                    f.write("Sector: = " + str(sector))
                    f.write('\n')
                    f.write("Parameters: " + str(param_grid))
                    f.write('\n')
                    f.write("accuracy = " + str(accuracy))
                    f.write('\n')
                    f.write("precision = " + str(precision))
                    f.write('\n')
                    f.write("recall = " + str(recall))
                    f.write('\n')
                    f.write("f1_score = " + str(f1_score))
                    f.write('\n')
                    f.write("f0.5_score = " + str(f0_5_score))
                    f.write('\n')
                    f.write("Best performing rule = " + str(tree))
                    f.write('\n')
                    f.write("Size training data: " + str(X_train.shape))
                    f.write('\n')
                    f.write("Size test data: " + str(X_test.shape))
                    f.write('\n')
                    f.write("Final training population stats: " + str(record))
    
                '''
                create tree diagram
                nodes, edges, labels = gp.graph(hof[0])
                
                g = pgv.Graph(format='png')
                g.add_nodes_from(nodes)
                g.add_edges_from(edges)
                g.layout(prog="dot")
                
                for i in nodes:
                    n = g.get_node(i)
                    n.attr["label"] = labels[i]
                
                g.write(path_img + "\{}_().pdf".format(sector, str(params)))
                '''